In [0]:
import pandas as pd
import numpy as np

# Task 1. Working with data

### 1.1. Download the data set movie_metadata.csv, which contains data about films from IMDb (Internet Movie Database)

In [0]:
# load data
df = pd.read_csv('movie_metadata.csv')
# get the first 5 rows
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0$,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0$,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0$,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0$,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,0.0$,NaN,12.0,7.1,NaN,0


### 1.2. The duration column contains data on the film length. How many missing values are there in this column?

In [0]:
df['duration'].isna().sum()

15

### 1.3. Replace the missing values in the duration column with the median value for this column.

In [0]:
df['duration'] = df['duration'].fillna(df['duration'].median())

### 1.4. What is the average film length? Give the answer as a floating-point figure rounded to two decimal places.

In [0]:
avg_film_length = df['duration'].mean()
print(f'Average film length is: {round(avg_film_length, 2)} mins')

Average film length is: 107.19 mins


### 1.5. Create a movie_duration_category column, which will contain three categories depending on the film length:
### • Category "1. <90" if the film is less than 90 minutes long

### • Category "2. 90–120" if the film is between 90 minutes and two hours long (inclusively)

### • Category "3. >120" if the film is more than two hours long





In [0]:
# function that maps duration to one of the three categories
def num_to_category(x):
    if x < 90:
        return '1. <90'
    elif 90 <= x <= 120:
        return '2. 90-120'
    return '3. >120'
# apply our function to the whole duration column
df['movie_duration_category'] = df['duration'].apply(num_to_category)
# look at first 5 values
df[['duration', 'movie_duration_category']][:5]

,duration,movie_duration_category
0,178.0,3. >120
1,169.0,3. >120
2,148.0,3. >120
3,164.0,3. >120
4,103.0,2. 90-120


### 1.6. Build a summary table for films released after 2000 (inclusively), to list the numbers of films:
### • Table rows: year
### • Table columns: movie duration category ("<90", "90–120", ">120")
### • The year of release should be displayed in the YYYY format.

In [0]:
df_year = df[df['title_year'] >= 2000].groupby(['title_year', 'movie_duration_category'])['movie_duration_category'].count().unstack()
# change index type from float to int
df_year.index = df_year.index.map(int)
# show summary table
df_year

movie_duration_category,1. <90,2. 90-120,3. >120
title_year,,,
2000,25,112,34
2001,29,120,39
2002,36,146,27
2003,31,108,30
2004,30,142,42
2005,31,142,48
2006,40,146,53
2007,31,130,43
2008,29,160,36


### 1.7. How many films between 90 minutes and two hours long were released in 2008?

In [0]:
df_year.loc[2008, '2. 90-120']

160

### 1.8. The plot_keywords column holds keywords characterizing the film's plot. Using the data in this column, create a column called movie_plot_category, to contain four categories depending on the key words in the column:
### • Category "love_and_death" if the keywords include both "love" and "death"
### • Category "love" if the keywords include the word "love"
### • Category "death" if the keywords include the word "death"
### • Category "other" if the keywords do not meet the conditions above

In [0]:
# function that maps keywords to category
def keyword_to_category(lst):
    if ('love' in lst) and ('death' in lst):
        return 'love_and_death'
    elif 'love' in lst:
        return 'love'
    elif 'death' in lst:
        return 'death'
    return 'other'

df['movie_plot_category'] = df['plot_keywords'].str.split('|').fillna('NaN').apply(keyword_to_category)   
df['movie_plot_category'].value_counts()

other             4722
love               189
death              123
love_and_death       9
Name: movie_plot_category, dtype: int64

### 1.9. The imdb_score column shows a viewer rating for the film. Build a table to reflect the average rating of films depending on which movie_plot_category category they belong to.

In [0]:
avg_rating = df.groupby('movie_plot_category')['imdb_score'].mean().to_frame('Average_rating')
avg_rating

,Average_rating
movie_plot_category,
death,6.508130
love,6.533862
love_and_death,6.677778
other,6.436298


### 1.10. What is the average rating of films in the "love" category? Give the answer as a floatingpoint figure rounded to two decimal places.

In [0]:
round(avg_rating.loc['love', 'Average_rating'], 2)

6.53

### 1.11. The budget column contains the film's budget. What is the median budget for all the films listed? Give the answer as an integer.

In [0]:
# strip $ sign and convert string to int
df['budget'] = df['budget'].map(lambda x: x.rstrip('$')).astype(float).astype(int)
int(df['budget'].median())

15000000

## Please indicate how long it took you to complete this task.
**~ 1h**

# Task 2. Problem solving

### 1. Download the event_data.csv dataset, which contains data on the use of the mobile application of users who registered from July 29 to September 1, 2019:
### • user_id - user identifier;
### • event_date - time of the event;
### • event_type - type of event: registration - registration in the application; simple_event - click event in the application; purchase - an event of purchase within the application;
### purchase_amount - purchase amount.

In [0]:
event_df = pd.read_csv('event_data.csv')
event_df.head()

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN


### 2. Highlight user cohorts based on the week of registration in the application. The cohort identifier should be the week ordinal (for example, the week from July 29 to August 4 should have identifier 31).

In [0]:
# convert event_date column to datetime type
event_df['event_date'] = pd.to_datetime(event_df['event_date'])
# create cohort_id column
event_df['cohort_id'] = event_df['event_date'].dt.week
event_df.head()

,user_id,event_date,event_type,purchase_amount,cohort_id
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


### 3. How many unique users in the cohort with ID 33?

In [0]:
event_df[event_df['cohort_id'] == 33]['user_id'].nunique()

5102

### 4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The lifetime indicator is calculated based on the serial number of the week in which the event is committed, relative to the week of registration. For example, an event committed on August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero week of lifetime, and an event committed by the same user on August 5 will be committed on the first week of lifetime).

In [0]:
# compute registration week id
reg_id = event_df.groupby('user_id')['cohort_id'].min()
# set column name
reg_id = reg_id.to_frame('first_visit')
# add column to event_df
event_df = pd.merge(event_df, reg_id, on='user_id')
# compute lifetime indicator
event_df['lifetime'] = event_df['cohort_id'] - event_df['first_visit']
event_df.head()

,user_id,event_date,event_type,purchase_amount,cohort_id,first_visit,lifetime
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,31,0
1,c40e6a,2019-07-29 21:13:24,simple_event,NaN,31,31,0
2,c40e6a,2019-07-30 22:20:50,simple_event,NaN,31,31,0
3,c40e6a,2019-08-01 13:13:01,simple_event,NaN,31,31,0
4,c40e6a,2019-08-02 11:19:11,simple_event,NaN,31,31,0


### 5. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.

In [0]:
visits = event_df.groupby(['first_visit', 'lifetime']).agg({'user_id': 'nunique'}).reset_index()
# new users
start_users = visits[visits['lifetime'] == 0][['first_visit', 'user_id']]
# set column names
start_users.columns = ['first_visit', 'start_users']
# merge two tables
visits = visits.merge(start_users)
# compute retention
visits['retention'] = visits['user_id'] / visits['start_users']
# create pivot table
retention_pivot = visits.pivot_table(index = 'first_visit', 
                                            columns = 'lifetime', values = 'retention', aggfunc = 'sum')
retention_pivot

lifetime,0,1,2,3,4
first_visit,,,,,
31,1.0,0.927595,0.629367,0.356962,0.15038
32,1.0,0.929303,0.648053,0.361168,NaN
33,1.0,0.924205,0.661125,NaN,NaN
34,1.0,0.929078,NaN,NaN,NaN
35,1.0,NaN,NaN,NaN,NaN


### 6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent, rounded to 2 decimal places, inclusive.

In [0]:
round(retention_pivot[3][32] * 100, 2)

36.12

### 7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying User) for cohorts depending on lifetime.

In [0]:
# number of users and amount paid for each group
arppu = event_df[event_df['purchase_amount'] > 0].groupby(['first_visit', 'lifetime']).agg({'purchase_amount': 'sum', 'user_id': 'count'}).reset_index()
# compute arppu
arppu['ARPPU'] = arppu['purchase_amount'] / arppu['user_id']
# create pivot table
arppu_pivot = arppu.pivot_table(index = 'first_visit', 
                                            columns = 'lifetime', values = 'ARPPU', aggfunc = 'sum')
arppu_pivot

lifetime,0,1,2,3,4
first_visit,,,,,
31,28.132911,29.511494,30.525000,29.142857,30.454545
32,30.222841,29.773692,29.928741,29.528302,NaN
33,30.689655,30.519663,29.171975,NaN,NaN
34,30.776699,30.929134,NaN,NaN,NaN
35,30.721925,NaN,NaN,NaN,NaN


### 8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point number, rounded to 2 decimal places, inclusive.

In [0]:
round(arppu_pivot[3][31], 2)

29.14

### 9. What is the median time between user registration and first purchase? Give the answer in seconds (!) As an integer.

In [0]:
# first visit date
first_visit = event_df.groupby('user_id')['event_date'].min()
# set column name
first_visit = first_visit.to_frame('first_visit_date')
# add new column to dataset
event_df = pd.merge(event_df, first_visit, on='user_id')
event_df.head()

,user_id,event_date,event_type,purchase_amount,cohort_id,first_visit,lifetime,first_visit_date
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,31,0,2019-07-29 00:02:15
1,c40e6a,2019-07-29 21:13:24,simple_event,NaN,31,31,0,2019-07-29 00:02:15
2,c40e6a,2019-07-30 22:20:50,simple_event,NaN,31,31,0,2019-07-29 00:02:15
3,c40e6a,2019-08-01 13:13:01,simple_event,NaN,31,31,0,2019-07-29 00:02:15
4,c40e6a,2019-08-02 11:19:11,simple_event,NaN,31,31,0,2019-07-29 00:02:15


In [0]:
#first purchse date
first_purchase = event_df[event_df['purchase_amount'] > 0].groupby('user_id')['event_date'].first()
# set column name
first_purchase = first_purchase.to_frame('first_purchase_date')
# add new column to dataset
event_df = pd.merge(event_df, first_purchase, on='user_id')
event_df.head()

,user_id,event_date,event_type,purchase_amount,cohort_id,first_visit,lifetime,first_visit_date,first_purchase_date
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,31,0,2019-07-29 00:02:15,2019-08-10 11:40:06
1,c40e6a,2019-07-29 21:13:24,simple_event,NaN,31,31,0,2019-07-29 00:02:15,2019-08-10 11:40:06
2,c40e6a,2019-07-30 22:20:50,simple_event,NaN,31,31,0,2019-07-29 00:02:15,2019-08-10 11:40:06
3,c40e6a,2019-08-01 13:13:01,simple_event,NaN,31,31,0,2019-07-29 00:02:15,2019-08-10 11:40:06
4,c40e6a,2019-08-02 11:19:11,simple_event,NaN,31,31,0,2019-07-29 00:02:15,2019-08-10 11:40:06


In [0]:
# compute differense
diff = event_df['first_purchase_date'] - event_df['first_visit_date']
# convert difference to seconds and compute median
median_diff = int(diff.dt.total_seconds().median())
median_diff

516788

## Please indicate how long it took you to complete this task.
**~ 2.5h**

# Task 4.
### You are given two random variables X and Y.
### E(X) = 0.5, Var(X) = 2
### E(Y) = 7, Var(Y) = 3.5
### cov (X, Y) = -0.8
### Find the variance of the random variable Z = 2X - 3Y

## **Solution:**
Var[X + Y] = E[(X + Y - E[X + Y])^2] = E[((X - E[X]) + (Y - E[Y]))^2] = 
= E[(X - E[X])^2 + (Y - E[Y])^2 + 2 * (X - E[X])* (Y - E[Y])] = 

= Var[X] + Var[Y] + 2 * cov(X,Y)

Recall that: Var[aX] = a^2 * Var[X] and cov(aX,bY) = a * b * cov(X,Y)

Var[2X-3Y] = Var[2X] + Var[-3Y] + cov(2X,-3Y) = 4 * Var[X] + 9 * Var[Y] -  2 *  2 * 3 * cov(X,Y) = 4 * 2 + 9 * 3.5 - 12 * (- 0.8) = 49.1

**Answer: 49.1**

# Task 5.
Omer trained a linear regression model and tested its performance on a test sample of 500
objects. On 400 of those, the model returned a prediction higher than expected by 0.5, and on
the remaining 100, the model returned a prediction lower than expected by 0.7.
What is the MSE for his model?
Limor claims that the linear regression model wasn't trained correctly, and we can do improve it
by changing all the answers by a constant value. What will be her MSE?
You can assume that Limor found the smallest error under her constraints.
Return two values - Omer's and Limor's MSE.

## **Solution:**
MSE(Omer) = 1/500 * ( 400 * (0.5)^2 + 100 * (0.7)^2) = 1/500 * 149 = 0.298

MSE(Limor) = 1/500 * ( 400 * (0.5 + c)^2 + 100 * (0.7 + c)^2)

Let's find c tham minimize MSE(Limor):

2 * 400 * (0.5 + c) + 2 * 100 * (0.7 + c) = 0

400 + 800 * c + 140 + 200 * c = 0

c = - 0.54

MSE(Limor) = 1/500 * ( 400 * (0.5 - 0.54)^2 + 100 * (0.7 - 0.54)^2) = 
1/500 * 3.2 = 0.0064

**Answer: MSE(Omer) = 0.298, MSE(Limor) = 0.0064**